# Import Library

In [2]:
from pydantic import BaseModel

class HouseFeatures(BaseModel):
    bedRoom: int
    bathroom: int
    area: float
    price_per_sqft: float
    floorNum: int

HouseFeatures(bedRoom=2, bathroom=2, area=1000, price_per_sqft=1000, floorNum=2).model_dump()

{'bedRoom': 2,
 'bathroom': 2,
 'area': 1000.0,
 'price_per_sqft': 1000.0,
 'floorNum': 2}

In [1]:
import pandas as pd , numpy as np 
from matplotlib import pyplot as plt
from plotly import express as px

# Loading dataset

In [10]:
import yaml, os

yaml_file_path = "../config/config.yaml"

with open(yaml_file_path) as yaml_file:
    config = yaml.safe_load(yaml_file)
    yaml_file.close()

config

{'dataset': {'train': 'artifact/dataset/train.csv',
  'test': 'artifact/dataset/test.csv',
  'raw': 'artifact/dataset/data.csv'},
 'preprocessor': {'path': 'artifact/preprocessor/preprocessor.pkl'},
 'model': {'path': 'artifact/model/model.pkl'}}

In [11]:
os.makedirs(os.path.dirname(config["preprocessor"]["path"]), exist_ok=True)

In [7]:
os.path.dirname(config["preprocessor"]["path"])

'artifact/preprocessor'

In [2]:
data_fp = "../artifact/dataset/data.csv"

df = pd.read_csv(data_fp)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           4600 non-null   object 
 1   price          4600 non-null   float64
 2   bedrooms       4600 non-null   float64
 3   bathrooms      4600 non-null   float64
 4   sqft_living    4600 non-null   int64  
 5   sqft_lot       4600 non-null   int64  
 6   floors         4600 non-null   float64
 7   waterfront     4600 non-null   int64  
 8   view           4600 non-null   int64  
 9   condition      4600 non-null   int64  
 10  sqft_above     4600 non-null   int64  
 11  sqft_basement  4600 non-null   int64  
 12  yr_built       4600 non-null   int64  
 13  yr_renovated   4600 non-null   int64  
 14  street         4600 non-null   object 
 15  city           4600 non-null   object 
 16  statezip       4600 non-null   object 
 17  country        4600 non-null   object 
dtypes: float

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,2014-05-02 00:00:00,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,2014-05-02 00:00:00,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,USA
2,2014-05-02 00:00:00,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,2014-05-02 00:00:00,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,2014-05-02 00:00:00,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,USA


In [1]:
from omegaconf import OmegaConf
from hydra.utils import instantiate  

# Load YAML config
model_config = OmegaConf.load("../config/model_config.yaml")  # Adjust path if needed

# Instantiate models with parameters
MODEL_CONFIGS = {}
for k, v in model_config.items():
    print(f"Instantiating model: {k}")
    MODEL_CONFIGS[k] = {
        'model': instantiate(v["model"]),     # v already contains _target_ and params
        'params': v["params"]
    }


Instantiating model: LinearRegression
Instantiating model: DescisionTreeRegressor
Instantiating model: RandomForestRegressor
Instantiating model: GradientBoostingRegressor
Instantiating model: AdaBoostRegressor
Instantiating model: CatBoostRegressor


In [2]:
MODEL_CONFIGS

{'LinearRegression': {'model': LinearRegression(), 'params': {'n_jobs': [-1]}},
 'DescisionTreeRegressor': {'model': DecisionTreeRegressor(),
  'params': {'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'], 'max_depth': [4, 6, 8, 10], 'n_estimators': [32, 64, 128, 256]}},
 'RandomForestRegressor': {'model': RandomForestRegressor(),
  'params': {'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'], 'max_depth': [4, 6, 8, 10], 'n_estimators': [32, 64, 128, 256]}},
 'GradientBoostingRegressor': {'model': GradientBoostingRegressor(),
  'params': {'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'], 'max_depth': [4, 6, 8, 10], 'n_estimators': [32, 64, 128, 256]}},
 'AdaBoostRegressor': {'model': AdaBoostRegressor(),
  'params': {'learning_rate': [0.1, 0.5, 0.01, 0.05, 0.001], 'n_estimators': [32, 64, 128, 256]}},
 'CatBoostRegressor': {'model': <catboost.core.CatBoostRegressor at 0x26a5c680f40>,
  'params': {'depth': [4

In [9]:
from omegaconf import OmegaConf
from hydra.utils import instantiate  # if using Hydra

conf = OmegaConf.load("../config/model_config.yaml")
model = instantiate(conf.models.LinearRegression)
print(type(model))

model

ConfigAttributeError: Missing key models
    full_key: models
    object_type=dict

In [22]:
conf.items()

dict_items([('models', {'LinearRegression': {'_target_': 'sklearn.linear_model.LinearRegression', 'n_jobs': [-1]}})])

In [16]:
for each in conf["models"]["LinearRegression"]['n_jobs']:
    print(each)

-1


In [23]:
from datetime import datetime
import os

def add_timestamp_to_filename(file_path: str) -> str:
    base, ext = os.path.splitext(file_path)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    return f"{base}_{timestamp}{ext}"

# Example usage:
new_path = add_timestamp_to_filename("artifact/model/report.csv")
print(new_path)


artifact/model/report_20250813_120032.csv


In [ ]:
for each in conf["models"].keys():
    model = instantiate(conf.models[each])

print
model

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,-1
,positive,False


In [8]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
print(type(model))
model

<class 'sklearn.linear_model._base.LinearRegression'>


,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [10]:
from omegaconf import OmegaConf
from hydra.utils import instantiate  

# Load YAML config
model_config = OmegaConf.load("../config/model_config.yaml")  # Adjust path if needed

# Instantiate models with parameters
MODEL_CONFIGS = {}
for k, v in model_config.items():
    print(f"Instantiating model: {k}")
    MODEL_CONFIGS[k] = {
        'model': instantiate(v["model"]),     # v already contains _target_ and params
        'params': v["params"]
    }


print(type(MODEL_CONFIGS["LinearRegression"]["model"]))

MODEL_CONFIGS["LinearRegression"]["model"]

Instantiating model: LinearRegression
Instantiating model: DescisionTreeRegressor
Instantiating model: RandomForestRegressor
Instantiating model: GradientBoostingRegressor
Instantiating model: AdaBoostRegressor
Instantiating model: CatBoostRegressor
<class 'sklearn.linear_model._base.LinearRegression'>


,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [3]:
df.nunique()

date               70
price            1741
bedrooms           10
bathrooms          26
sqft_living       566
sqft_lot         3113
floors              6
waterfront          2
view                5
condition           5
sqft_above        511
sqft_basement     207
yr_built          115
yr_renovated       60
street           4525
city               44
statezip           77
country             1
dtype: int64

In [7]:
df.columns

Index(['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'street', 'city',
       'statezip', 'country'],
      dtype='object')

In [4]:
df.isna().sum()

date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
street           0
city             0
statezip         0
country          0
dtype: int64

In [6]:
df['statezip'].value_counts()

statezip
WA 98103    148
WA 98052    135
WA 98117    132
WA 98115    130
WA 98006    110
           ... 
WA 98047      6
WA 98288      3
WA 98050      2
WA 98354      2
WA 98068      1
Name: count, Length: 77, dtype: int64

# Data Cleaning